In [1]:
import pandas as pd

In [22]:
import re

## Get the GEODE taxa list into a dataframe

In [2]:
geode_taxa_list_csv = '/nobackup/scratch/grp/grp_geode/scripts_and_files/prepare_data/geode_taxa_list.csv'
geode_taxa_list = pd.read_csv(geode_taxa_list_csv, sep=',', header=None, names=['names', 'sequence_id'])

In [3]:
names = geode_taxa_list['names'].str.split(pat='_', expand=True)
geode_taxa_list[['geode_id', 'family', 'genus', 'species']] = names

## Find the probe set, plate, and fastq

In [4]:
from pathlib import Path
import itertools
import sys

In [29]:
def split_path_values(sequence_id):
    raw_sequence_dir = '/nobackup/scratch/grp/grp_geode/raw_sequences'
    full_path = str(list(itertools.islice(Path(raw_sequence_dir).rglob(f"*{sequence_id}*"), 1))[0])
    full_path_split = full_path.split('/')
    folder = '/'.join(full_path_split[:-1])
    try:
        plate = re.search(r'/(plate\d{1,3})/', full_path).group(1)
    except AttributeError:
        plate = ''
    probe_set = re.search(r'/(\d{2,3}kb)/', full_path).group(1)
    fastq = full_path_split[-1].replace('_R1_001.fastq.gz', '')
    return (folder, plate, probe_set, fastq)

In [26]:
print(split_path_values('105602_P001_WA01'))
print(split_path_values('105603_P001_WA01'))

('/nobackup/scratch/grp/grp_geode/raw_sequences/500kb/plate1', 'plate1', '500kb', '105602_P001_WA01_R2.fastq.gz')
('/nobackup/scratch/grp/grp_geode/raw_sequences/20kb/plate01/fastqs', 'plate01', '20kb', 'RAPiD-Genomics_F320_BYU_105603_P001_WA01_i5-535_i7-59_S337_L001_R2_001.fastq.gz')


In [30]:
split_path_result = geode_taxa_list['sequence_id'].map(split_path_values)

In [31]:
geode_taxa_list[['folder', 'plate', 'probe_set', 'fastq']] = pd.DataFrame(split_path_result.to_list())

In [32]:
geode_taxa_list[(geode_taxa_list['probe_set'] != '20kb') & (geode_taxa_list['probe_set'] != '500kb')]

,names,sequence_id,geode_id,family,genus,species,folder,plate,probe_set,fastq


In [33]:
geode_taxa_list

,names,sequence_id,geode_id,family,genus,species,folder,plate,probe_set,fastq
0,GEODE7884_Euphaeidae_Euphaea_guerini,105603_P001_WA01,GEODE7884,Euphaeidae,Euphaea,guerini,/nobackup/scratch/grp/grp_geode/raw_sequences/...,plate01,20kb,RAPiD-Genomics_F320_BYU_105603_P001_WA01_i5-53...
1,GEODE7885_Euphaeidae_Euphaea_refulgens,105603_P001_WA02,GEODE7885,Euphaeidae,Euphaea,refulgens,/nobackup/scratch/grp/grp_geode/raw_sequences/...,plate01,20kb,RAPiD-Genomics_F320_BYU_105603_P001_WA02_i5-53...
2,GEODE7886_Euphaeidae_Euphaea_splendens,105603_P001_WA03,GEODE7886,Euphaeidae,Euphaea,splendens,/nobackup/scratch/grp/grp_geode/raw_sequences/...,plate01,20kb,RAPiD-Genomics_F320_BYU_105603_P001_WA03_i5-53...
3,GEODE7887_Euphaeidae_Euphaea_ornata,105603_P001_WA04,GEODE7887,Euphaeidae,Euphaea,ornata,/nobackup/scratch/grp/grp_geode/raw_sequences/...,plate01,20kb,RAPiD-Genomics_F320_BYU_105603_P001_WA04_i5-53...
4,GEODE7888_Euphaeidae_Euphaea_inouei,105603_P001_WA05,GEODE7888,Euphaeidae,Euphaea,inouei,/nobackup/scratch/grp/grp_geode/raw_sequences/...,plate01,20kb,RAPiD-Genomics_F320_BYU_105603_P001_WA05_i5-53...
...,...,...,...,...,...,...,...,...,...,...
3090,GEODE18565_Corduliidae_Hemicordulia_chrysochlora,105603_P022_WH08,GEODE18565,Corduliidae,Hemicordulia,chrysochlora,/nobackup/scratch/grp/grp_geode/raw_sequences/...,plate22,20kb,RAPiD-Genomics_F391_BYU_105603_P022_WH08_i5-11...
3091,GEODE18566_Corduliidae_Procordulia_papandayane...,105603_P022_WH09,GEODE18566,Corduliidae,Procordulia,papandayanensis,/nobackup/scratch/grp/grp_geode/raw_sequences/...,plate22,20kb,RAPiD-Genomics_F391_BYU_105603_P022_WH09_i5-11...
3092,GEODE18057B_Synthemistidae_Apocordulia_macrops,105603_P022_WH10,GEODE18057B,Synthemistidae,Apocordulia,macrops,/nobackup/scratch/grp/grp_geode/raw_sequences/...,plate22,20kb,RAPiD-Genomics_F391_BYU_105603_P022_WH10_i5-11...
3093,GEODE18266_Synthemistidae_Choristhemis_olivei,105603_P022_WH11,GEODE18266,Synthemistidae,Choristhemis,olivei,/nobackup/scratch/grp/grp_geode/raw_sequences/...,plate22,20kb,RAPiD-Genomics_F391_BYU_105603_P022_WH11_i5-11...


## Put dataframe into sqlite table

In [34]:
import sqlite3

In [35]:
sqlite_db = '/nobackup/scratch/grp/grp_geode/payton/nextflow/geode.db'
con = sqlite3.connect(sqlite_db)

geode_taxa_list.to_sql('taxon', con, if_exists='replace', index=False)

con.commit()
con.close()